In [ ]:
# DL Assignment 3
# version 19.
# code to run best model.

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split


# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:

# path = '/kaggle/input/marathi/mar_train.csv'
# path_val = '/kaggle/input/marathi/mar_valid.csv'
# path_test = '/kaggle/input/marathi/mar_test.csv'
path = '/content/drive/MyDrive/Colab Notebooks/DL Assignment/Assignment 3/aksharantar_sampled/aksharantar_sampled/mar/mar_train.csv'
path_val = '/content/drive/MyDrive/Colab Notebooks/DL Assignment/Assignment 3/aksharantar_sampled/aksharantar_sampled/mar/mar_valid.csv'
path_test = '/content/drive/MyDrive/Colab Notebooks/DL Assignment/Assignment 3/aksharantar_sampled/aksharantar_sampled/mar/mar_test.csv'

In [ ]:
df = pd.read_csv(path , header = None)
df_val = pd.read_csv(path_val , header = None)
df_test = pd.read_csv(path_test , header = None)
english_words_val = df_val[0]
marathi_words_val = df_val[1]
english_words_test = df_test[0]
marathi_words_test = df_test[1]
english_words = df[0]
marathi_words = df[1]

                         0                 1
50000  audhogikikaranamule   औधोगिकीकरणामुळे
50001     anyayaviruddhahi  अन्यायाविरुद्धही
50002              fairade            फैराडे
50003          inputmadhye        इनपुटमध्ये
50004      charitranmadhil      चरित्रांमधील
...                    ...               ...
51195           chikateche           चिकटेचे
51196     anubhavanyachahi     अनुभवण्याचाही
51197            andhapuri          अंधापुरी
51198           ghadamodee           घडामोडी
51199         atiprachalit        अतिप्रचलित

[1200 rows x 2 columns]


In [ ]:
# creating list of charecters in both languages

english_char_list = []
max_length_word_english = -1
for word in english_words:
  max_length_word_english = max(max_length_word_english,len(word)) 
  for char in word :
    english_char_list.append(char);
english_char_list = list(set(english_char_list))
english_char_list.sort()

marathi_char_list = []
max_length_word_marathi = -1
for word in marathi_words:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
  for char in word :
    marathi_char_list.append(char);
marathi_char_list = list(set(marathi_char_list))
marathi_char_list.sort()


# finding out the maximum size word for english and marathi from validation and test data.
for word in english_words_val:
  max_length_word_english = max(max_length_word_english,len(word))
for word in english_words_test:
  max_length_word_english = max(max_length_word_english,len(word)) 
for word in marathi_words_val:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
for word in marathi_words_test:
  max_length_word_marathi = max(max_length_word_marathi,len(word))


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऍ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्']
28
20


In [ ]:
# english word to vector size = 27 ie. max_length_word_english
# marathi word to vector size = 20 ie. max_length_word_marathi
# for one word.
def word2vec(word, lang):
  vec = []
  if(lang == "english"):
    vec.append(len(english_char_list) + 1)
    for char in word:
      for i in range(len(english_char_list)):
        if(english_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_english + 1): # padding with max_length + 1.
        vec.append(0)
    vec.append(0)
  else :
    vec.append(len(marathi_char_list) + 1)
    for char in word:
      for i in range(len(marathi_char_list)):
        if( marathi_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_marathi + 1):  # padding with max_length + 1.
        vec.append(0)
    vec.append(0)
  return(vec)


In [ ]:
vec = word2vec(marathi_words[10],"marathi")

हरिहरक्षेत्र
[64, 49, 42, 52, 49, 42, 17, 63, 47, 58, 31, 63, 42, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# creating matrix of representation for whole words of english and marathi.

def ip_matrix_construct(words, lang):
  ans = []
  for word in words:
    ans.append(word2vec(word, lang))
  return(ans)

In [ ]:
# calculated representations of whole english and marathi words in variables english and marathi matrix.
english_matrix = ip_matrix_construct(english_words, "english")
marathi_matrix = ip_matrix_construct(marathi_words, "marathi")
english_matrix = torch.tensor(english_matrix)
marathi_matrix = torch.tensor(marathi_matrix)

english_matrix_val = ip_matrix_construct(english_words_val, "english")
marathi_matrix_val = ip_matrix_construct(marathi_words_val, "marathi")
english_matrix_val = torch.tensor(english_matrix_val)
marathi_matrix_val = torch.tensor(marathi_matrix_val)
english_matrix_test = ip_matrix_construct(english_words_test, "english")
marathi_matrix_test =ip_matrix_construct(marathi_words_test, "marathi")
english_matrix_test = torch.tensor(english_matrix_test)
marathi_matrix_test = torch.tensor(marathi_matrix_test)

51200
51200
30
22


In [ ]:

class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, enc_layers, p, cell_type, bidirectional):
    super(Encoder,self).__init__()
    self.hidden_size = hidden_size
    self.enc_layers = enc_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
      output, hidden = self.gru(embedding)
    if(self.cell_type == "RNN"):
      output, hidden = self.rnn(embedding)
    if(self.cell_type == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      return outputs, hidden, cell
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:

class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
  
  def forward(self,x,output, hidden, cell = 0):
    x = x.unsqueeze(0).int()
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(embedding, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(embedding, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
    # shape of outputs: (1, N, hidden_size)
    predictions = self.fc(outputs)
    # shape of predictions: (1, N, length_of_vocab)
    predictions = predictions.squeeze(0)
    # shape of predictions: (N, length_of_vocab)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden


  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class Atten_decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type, bidirectional):
    super(Atten_decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.max_length = len(english_matrix[0])  
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(hidden_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
    self.attn = nn.Linear(hidden_size+embedding_size, self.max_length)
    if(bidirectional):
      self.attn_combine = nn.Linear(hidden_size * 2 + embedding_size, hidden_size)
    else :
      self.attn_combine = nn.Linear(hidden_size + embedding_size, hidden_size)

  def forward(self, x,output, hidden, cell = 0):
    x = x.unsqueeze(0)
    output=output.permute(1,0,2)
    embedded = self.embedding(x)
    embedded = self.dropout(embedded)
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0],hidden[0]), 1)), dim = 1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(1),output)
    attn_applied = attn_applied.squeeze(1)
    op = torch.cat((embedded[0], attn_applied), 1)

    op = self.attn_combine(op).unsqueeze(0)
    op = F.relu(op)
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(op, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(op, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(op, (hidden, cell))
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden



In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, enc_layers, dec_layers):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(marathi_char_list) + 2  
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        if(self.cell_type == "LSTM"):
            encoder_output, hidden, cell = self.encoder(source)
        else:
            encoder_output, hidden = self.encoder(source)
        if(self.enc_layers != self.dec_layers or self.bidirectional == True):
          hidden = hidden[self.enc_layers - 1] + hidden[self.enc_layers - 1]
          hidden = hidden.repeat(self.dec_layers,1,1)
          if(self.cell_type == "LSTM"):
              cell = cell[self.enc_layers - 1] + cell[self.enc_layers - 1]
              cell = cell.repeat(self.dec_layers,1,1)
        
        x = target[0]
    
        for t in range(1, target_len):
            if(self.cell_type == "LSTM"):
                output, hidden, cell = self.decoder(x, encoder_output, hidden, cell)
            else :
                output, hidden = self.decoder(x, encoder_output, hidden)
            outputs[t] = output

            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess
            
        return outputs


In [ ]:
def Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size):
    correct_count = 0
    for batch_idx in range((int)(len(english_matrix) / batch_size)):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(batch_size):
            if(torch.equal(output[i][1:],target[i][1:])):
                correct_count += 1
    accuracy = correct_count * 100 / len(english_matrix)
    return accuracy
        

In [ ]:
# creating list of expected marathi word and predicted marathi word.
predictions_vanilla_train = []
predictions_vanilla_val = []
predictions_vanilla_test = []
def matrix_to_words(model, english_matrix, marathi_matrix, batch_size, data_type):
  for batch_idx in range((int)(len(english_matrix) / batch_size)):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(len(target)):
          target_word = target[i]
          output_word = output[i]
          word1 = ""
          word2 = ""
          for j in range(len(target_word)):
            if(target_word[j]>0 and target_word[j]<64):
              word1 += marathi_char_list[target_word[j] - 1]
          for j in range(len(output_word)):
            if(output_word[j]>0 and output_word[j]<64):
              word2 += marathi_char_list[output_word[j] - 1]
          temp = [word1, word2]
          if(data_type == "train"):
            predictions_vanilla_train.append(temp)
          if(data_type == "validation"):
            predictions_vanilla_val.append(temp)
          if(data_type == "test"):
            predictions_vanilla_test.append(temp)

In [ ]:
def neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention):
  learning_rate = 1e-3
  num_epochs = 20
  input_size_encoder = len(english_char_list) + 2  
  input_size_decoder = len(marathi_char_list) + 2  
  output_size        = len(marathi_char_list) + 2  

  encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, enc_layers, enc_dropout, cell_type,bidirectional).to(device)
  if(attention):
    decoder_net = Atten_decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type, bidirectional).to(device)
  else:
    decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type).to(device)

  model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, enc_layers, dec_layers).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  pad_idx = len(marathi_char_list) + 1  # 64 # pading index for marathi
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


  for epoch in range(num_epochs):
      print("epoch = ",epoch)

      model.train()
      total_loss = 0
      step = 0
      for batch_idx in range((int)(len(english_matrix) / batch_size)):

          inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
          target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
          target = target.T
          output = model(inp_data.T, target)

          output = output[1:].reshape(-1, output.shape[2])
          target = target[1:].reshape(-1)
          optimizer.zero_grad()
          loss = criterion(output, target)
          total_loss += loss
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

          optimizer.step()

          step += 1
      print("total loss = ",total_loss / step)
      training_accuracy = Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size)
      print("Training Accuracy = ", training_accuracy)
      val_accuracy = Accuracy(model, english_matrix_val, marathi_matrix_val, epoch, batch_size)
      print("Validation accuracy = ",val_accuracy)
      test_accuracy = Accuracy(model, english_matrix_test, marathi_matrix_test, epoch, batch_size)
      print("Test accuracy = ", test_accuracy)
      if(epoch > 10 and training_accuracy < 10):
        return
      if(epoch > 5 and training_accuracy < 1):
        return
  matrix_to_words(model, english_matrix, marathi_matrix, batch_size, "train")
  print("train done")
  matrix_to_words(model, english_matrix_val, marathi_matrix_val, batch_size, "validation")
  print("val done")
  matrix_to_words(model, english_matrix_test, marathi_matrix_test, batch_size, "test")

In [ ]:
cell_type = "GRU"
bidirectional = False
enc_layers = 5
dec_layers = 5
batch_size = 256
embedding_size = 256
hidden_size = 512
enc_dropout = 0.2
dec_dropout = 0.2
attention = False


neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention)

epoch =  0
total loss =  tensor(1.6752, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  0.0
Validation accuracy =  0.0244140625
Test accuracy =  0.0244140625
epoch =  1
total loss =  tensor(1.0414, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  6.099609375
Validation accuracy =  9.9853515625
Test accuracy =  8.642578125
epoch =  2
total loss =  tensor(0.5259, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  17.62890625
Validation accuracy =  20.458984375
Test accuracy =  18.26171875
epoch =  3
total loss =  tensor(0.3664, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  27.26953125
Validation accuracy =  26.0986328125
Test accuracy =  23.5107421875
epoch =  4
total loss =  tensor(0.3011, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  33.462890625
Validation accuracy =  28.80859375
Test accuracy =  25.48828125
epoch =  5
total loss =  tensor(0.2513, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  42.1562

In [ ]:
# creating csv file of predictions
filename_train  = 'predictions_vanilla_train.csv'
filename_val = 'predictions_vanilla_val.csv'
filename_test = 'predictions_vanilla_test.csv'

with open(filename_train, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_train)

print(f'CSV file "{filename_train}" has been created.')


with open(filename_val, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_val)

print(f'CSV file "{filename_val}" has been created.')


with open(filename_test, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_test)

print(f'CSV file "{filename_test}" has been created.')

CSV file "predictions_vanilla_train.csv" has been created.
CSV file "predictions_vanilla_val.csv" has been created.
CSV file "predictions_vanilla_test.csv" has been created.
